## Description : 
#### Sample Notebook to read and write using amorphic utils -Pyspark



Load the pysaprk amorphic utils you created

copy the path from "Home -> Transformation -> ETL Library -> Details -> (hover over package path and click copy - use the version you need)"


In [1]:
%extra_py_files s3://nvd-us-west-2-816069158041-test-etl/common-libs/fe4b4d47-ca11-45e8-8030-1c2473c90abb/libs/python/amorphicutils-0.3.1.zip
    

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.7 and you have 1.0.0 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Extra py files to be included:
s3://nvd-us-west-2-816069158041-test-etl/common-libs/fe4b4d47-ca11-45e8-8030-1c2473c90abb/libs/python/amorphicutils-0.3.1.zip
Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 5
Session ID: 065933fa-3d6d-4983-a10d-026b927c2a50
Applying the following default arguments:
--glue_kernel_version 1.0.0
--enable-glue-datacatalog true
--extra-py-files s3://nvd-us-west-2-816069158041-test-etl/common-libs/fe

### Import all the essential packages

In [2]:
import sys
import time
import pandas as pd
import logging
from awsglue.utils import getResolvedOptions
from pyspark.sql.functions import lit, col, concat
from amorphicutils.common import read_param_store
from amorphicutils.pyspark.infra.gluespark import GlueSpark
from amorphicutils.pyspark import read
from amorphicutils.pyspark import write

JOB_NAME not available as args.


In [5]:
logging.basicConfig(level="INFO")
LOGGER = logging.getLogger()
LOGGER.setLevel(logging.INFO)

In [6]:
glue_spark = GlueSpark()
glue_context = glue_spark.get_glue_context()
spark = glue_spark.get_spark()

JOB_NAME not available as args.


In [7]:
def get_param_value(param_store_key, secure=False):
    """Function to read parameters from parameters store"""
    LOGGER.info(
        "In get_param_value, Fetching values stored  in %s, IsSecure : %s",
        param_store_key,
        secure,
    )
    param_value_response = read_param_store(param_store_key, secure=secure)
    if param_value_response["exitcode"] == 1:
        LOGGER.error(
            "Failed to get value for parameter {} with error {}.".format(
                param_store_key, param_value_response["message"]
            )
        )
        raise Exception(
            "Failed to get value for parameter {} with error {}.".format(
                param_store_key, param_value_response["message"]
            )
        )
    else:
        return param_value_response["data"]

### Configure below Variables

In [9]:
user_id = "harsha"  # Your Login UserId with access to write

# Replace below if you have different Domain to to Read from
r_domain = "bronzehealth"
# Replace this if you have different Domain to write into.
w_domain = "silverhealth"
# Replace this if you created new Dataset to Read from.
r_dataset = "Immunization" 
# Replace this if you created new Dataset to write into.
w_dataset = "Immunization_Staged" 

In [10]:
# LZ Bucket is Output Bucket  # DLZ Bucket is Input Bucket

lz_bucket = get_param_value(param_store_key="SYSTEM.S3BUCKET.LZ")
dlz_bucket = get_param_value(param_store_key="SYSTEM.S3BUCKET.DLZ")

my_param = get_param_value(
    param_store_key="nvd-sample-parm-harsha", secure=True
)  # Replace param_store_key with your paramstore name
print(my_param)

This my secure parameter store value


#### Instantiate Amorphic Reader and Writer Class

In [11]:
amporphic_reader = read.Read(dlz_bucket, spark)
amorphic_writer = write.Write(lz_bucket, glue_context)

## Read Dataset

In [13]:
response = amporphic_reader.read_csv_data(r_domain, r_dataset, header=True)
print(response['message'])


Successfully loaded s3 data into spark dataframe


In [18]:
if response["exitcode"] == 0:
    df_data = response["data"]
    df_data.select('client_id','client_first_name','status').show(5,truncate=False)

+------------------------------------+-----------------+---------+
|client_id                           |client_first_name|status   |
+------------------------------------+-----------------+---------+
|00901a88-59b2-44cc-a0b0-9aa7e8842487|Saltzman327      |Completed|
|03b86e36-59f5-4db3-af08-91231dbfdaf6|Washington743    |Completed|
|069335e0-addd-4969-b76e-3b8845474f54|Johnson823       |Completed|
|06e441e3-2ed3-4f08-b308-b2896aa78118|Nguyen378        |Completed|
|07d57b08-d37f-4514-a701-3c2f76a4e061|Numakair234      |Completed|
+------------------------------------+-----------------+---------+
only showing top 5 rows


### Transformation
##### Derive new column - client_full_name

In [19]:
df_copy = df_data.withColumn(
        "client_full_name",
        concat(col("client_first_name"), lit(" "), col("client_last_name")),
    ).withColumn(
        "Address",
        concat(
            lit("Apt "),
            col("apartment").cast("string"),
            lit(" St "),
            col("street_number").cast("string"),
            lit(" "),
            col("primary_direction"),
            lit(" "),
            col("street_name"),
            lit(" "),
            col("street_type"),
            lit(" "),
            col("secondary_direction"),
            lit(" Post "),
            col("po_box").cast("string"),
            lit(" "),
            col("city"),
            col("state"),
            lit(" ZIP: "),
            col("zip_code").cast("string"),
        ),
    )

In [21]:
# Drop unwanted columns
drop_col_list = ["client_first_name", "client_last_name", "client_suffix", "client_age", "client_alias_last_name", "client_alias_first_name", "client_alias_middle_name", "client_marital_status", "primary_race", "race_2", "race_3", "race_4", "race_5", "ethnicity", "client_ssn", "primary_care_first_name", "primary_care_middle_name", "primary_care_last_name", "primary_care_suffix", "patient_program_close_date", "patient_program_close_reason", "mother_maiden_name", "patient_notes", "street_number", "primary_direction", "street_name", "street_type", "secondary_direction", "po_box", "apartment", "city", "state", "zip_code", "primary_contact_last_name", "primary_contact_first_name", "primary_contact_relationship_type", "primary_contact_home_phone", "primary_contact_cell_phone", "primary_contact_work_phone", "patient_precautions_contraindications_type", "precautions_contraindications_effective_date", "lot_number", "administration_site", "observation_period", "reaction", "status", "administered_indicator", "administered_by", "administered_at", "dose_sequence"]
df_copy = df_copy.drop(*drop_col_list)

### Write to Dataset

In [23]:
response = amorphic_writer.write_csv_data(
        df_copy.coalesce(1), w_domain, w_dataset, user=user_id, full_reload=False
    )
print(response['message'])

Successfully saved data to s3.


In [4]:
%stop_session

Stopping session: 065933fa-3d6d-4983-a10d-026b927c2a50
Stopped session.
